<a href="https://colab.research.google.com/github/chungchihhan/https-github.com-kszuyen-A-Dating-App-for-Dogs/blob/main/llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install llama-index

In [14]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import os

# Google 雲端硬碟的根目錄路徑
path = "/content/drive/My Drive"

# 列出指定路徑下的所有檔案和資料夾
# files = os.listdir(path)
# for file in files:
#     print(file)


In [20]:
filepath = '/content/drive/MyDrive/TS31103_Adjustment_of_Backlash_for_GXA_S_Series_converted.txt'

In [29]:
#get the openai_api_key
from google.colab import userdata
userdata.get('OPENAI_API_KEY')


#use llama_index
from llama_index import VectorStoreIndex, SimpleDirectoryReader

reader = SimpleDirectoryReader(
    input_files=[filepath]
)
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")
# documents = SimpleDirectoryReader(filepath).load_data()
# index = VectorStoreIndex.from_documents(documents)

index = VectorStoreIndex.from_documents(docs)

Loaded 1 docs


In [30]:
query_engine = index.as_query_engine()

result = query_engine.query("背隙調整步驟")
response_text = result.response

print(response_text)

KeyboardInterrupt: 